In [10]:
import cv2
import easyocr
import re
import numpy as np
from paddleocr import PaddleOCR

# Paddle OCR

In [11]:
def readPlateTextPaddleOCR(image, ocr):
    result = ocr.ocr(image, cls=False)
    
    maxArea = 0
    largestText = 'No Detection'
    largestTextConfidence = 0
    
    # Loop through detections
    for idx in range(len(result)):
        res = result[idx]
        for bbox, detect in res:
    
            # Get box Coords
            x1, y1 = bbox[0]
            x2, y2 = bbox[-2]
            
            # Calculate area
            area = (x2 - x1) * (y2 - y1)
            
            # Update area if needed
            if area > maxArea:
                maxArea = area
                largestText = detect[0].upper()
                largestTextConfidence = detect[1]
    
        # Remove non-alphanumeric characters
        cleanText = re.sub(r'[^a-zA-Z0-9]', '', largestText)
    return cleanText, largestTextConfidence

## Easy OCR

In [12]:
def readPlateTextEasyOCR(image, reader):
    result = reader.readtext(image)
    
    maxArea = 0
    largestText = 'No Detection'
    largestTextConfidence = 0
    
    # Loop through detections
    for idx in range(len(result)):
        res = result[idx]
        bbox = res[0]
        detect = res[1]
        conf = res[2]
    
        # Get box Coords
        x1, y1 = bbox[0]
        x2, y2 = bbox[-2]
        
        # Calculate area
        area = (x2 - x1) * (y2 - y1)
        
        # Update area if needed
        if area > maxArea:
            maxArea = area
            largestText = detect.upper()
            largestTextConfidence = conf
    
    # Remove non-alphanumeric characters
    cleanText = re.sub(r'[^a-zA-Z0-9]', '', largestText)
    return cleanText, largestTextConfidence  

## Perform Tresholding

In [18]:
def imageThreshold(image):
    # Grayscale
    grayImage = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    
    # Resize the image to 4x
    # scale = 4
    # resizedImage = cv2.resize(grayImage, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    
    # Otsu thresholding
    _, plateThresh = cv2.threshold(grayImage, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    plateThresh = cv2.morphologyEx(plateThresh, cv2.MORPH_CLOSE, kernel)
    
    # Adaptive Threshold
    # plateThresh = cv2.adaptiveThreshold(
    #     plateThresh, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    #     cv2.THRESH_BINARY, 5, 10)
    
    return plateThresh

## Comparing Paddle OCR to Easy OCR

In [14]:
# Setup OCRs
paddleOCR = PaddleOCR(lang='en')
easyOCR = easyocr.Reader(['en'])

[2025/05/15 19:03:37] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Trever/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Trever/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_nu

In [19]:
# Load image
imageName = 'test1.jpg'
plateImage = cv2.imread(f'test-media/{imageName}')

# Perform thresholding
imageThresh = imageThreshold(plateImage)

# Save thresholding results
cv2.imwrite(f'test-media/{imageName.replace('.jpg', '')}Thresh.jpg', imageThresh)

# Read the text and return largest text area
plateTextEasyOCR, confidenceEasyOCR = readPlateTextEasyOCR(imageThresh, easyOCR)
plateTextPaddleOCR, confidencePaddleOCR = readPlateTextPaddleOCR(imageThresh, paddleOCR)

# Display resultsq
print('Easy OCR Results:')
print(f'Plate Text: {plateTextEasyOCR}')
print(f'Confidence: {confidenceEasyOCR}\n')
print('Paddle OCR Results:')
print(f'Plate Text: {plateTextPaddleOCR}')
print(f'Confidence: {confidencePaddleOCR}\n')
cv2.imshow('Image Threshold', imageThresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

[2025/05/15 19:04:28] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.009554147720336914
[2025/05/15 19:04:28] ppocr DEBUG: rec_res num  : 3, elapsed : 0.03430938720703125
Easy OCR Results:
Plate Text: XITCKZH
Confidence: 0.32806405186070764

Paddle OCR Results:
Plate Text: X17KZH
Confidence: 0.9568066000938416
